In [1]:
import pandas as pd

In [6]:
df_enroll = pd.read_csv("data/kddcup15/train/enrollment_train.csv")
df_log = pd.read_csv("data/kddcup15/train/log_train.csv")
df_truth = pd.read_csv("data/kddcup15/train/truth_train.csv")

In [7]:
df_enroll.head()

,enrollment_id,username,course_id
0,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila
1,3,1qXC7Fjbwp66GPQc6pHLfEuO8WKozxG4,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx
2,4,FIHlppZyoq8muPbdVxS44gfvceX9zvU7,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila
3,5,p1Mp7WkVfzUijX0peVQKSHbgd5pXyl4c,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx
4,6,dpK33RH9yepUAnyoywRwBt1AJzxGlaja,AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd


In [8]:
df_log.head()

,enrollment_id,time,source,event,object
0,1,2014-06-14T09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
1,1,2014-06-14T09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
2,1,2014-06-14T09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4
3,1,2014-06-14T09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W
4,1,2014-06-14T09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf


In [10]:
df_date = pd.read_csv("data/kddcup15/date.csv")
df_object = pd.read_csv("data/kddcup15/object.csv")

In [11]:
df_date.head()

,course_id,from,to
0,bWdj2GDclj5ofokWjzoa5jAwMkxCykd6,2014-05-26,2014-06-24
1,RXDvfPUBYFlVdlueBFbLW0mhhAyGEqpt,2014-05-25,2014-06-23
2,fbPkOYLVPtPgIt0MxizjfFJov3JbHyAi,2014-01-17,2014-02-15
3,A3fsA9Zfv1X2fVEQhTw51lKENdNrEqT3,2014-05-28,2014-06-26
4,5X6FeZozNMgE2VRi3MJYjkkFK8SETtu2,2014-06-09,2014-07-08


In [28]:
pd.to_datetime(df_date['from'])

0    2014-05-26
1    2014-05-25
2    2014-01-17
3    2014-05-28
4    2014-06-09
5    2013-12-11
6    2014-05-26
7    2013-11-01
8    2014-05-25
9    2014-06-19
10   2014-06-11
11   2013-10-30
12   2013-12-11
13   2014-05-13
14   2014-05-29
15   2014-05-19
16   2014-05-29
17   2014-05-28
18   2013-12-02
19   2014-05-26
20   2013-11-25
21   2013-12-12
22   2014-06-04
23   2014-05-25
24   2013-11-26
25   2014-05-26
26   2014-05-28
27   2014-05-20
28   2014-06-12
29   2013-10-27
30   2014-05-25
31   2014-05-29
32   2013-12-17
33   2014-06-15
34   2014-05-19
35   2014-07-03
36   2013-11-11
37   2013-11-28
38   2014-05-29
Name: from, dtype: datetime64[ns]

In [55]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score


In [82]:
def load_data(data_type='train'):
    """Load the required datasets."""
    base_path = f'data/kddcup15/{data_type}/'
    enrollment = pd.read_csv(f'{base_path}enrollment_{data_type}.csv')
    log = pd.read_csv(f'{base_path}log_{data_type}.csv')
    truth = pd.read_csv(f'{base_path}truth_{data_type}.csv', header=None, names=['enrollment_id', 'dropout'])
    return enrollment, log, truth

def load_additional_data():
    """Load date and object data."""
    date_df = pd.read_csv('data/kddcup15/date.csv')
    object_df = pd.read_csv('data/kddcup15/object.csv')
    return date_df, object_df


In [83]:
def preprocess_enrollment(enrollment):
    """Preprocess the enrollment data."""
    # Create copies of username and course_id for encoding
    #enrollment['username_encoded'] = enrollment['username']
    #enrollment['course_id_encoded'] = enrollment['course_id']
    
    #le = LabelEncoder()
    #enrollment['username_encoded'] = le.fit_transform(enrollment['username_encoded'])
    #enrollment['course_id_encoded'] = le.fit_transform(enrollment['course_id_encoded'])
    return enrollment


def process_date_data(date_df, enrollment_df):
    """Process date data and merge with enrollment data."""
    date_df['from'] = pd.to_datetime(date_df['from'])
    date_df['to'] = pd.to_datetime(date_df['to'])
    date_df['course_duration'] = (date_df['to'] - date_df['from']).dt.days

    return pd.merge(enrollment_df, date_df[['course_id', 'course_duration']], on='course_id', how='left')

def process_object_data(object_df, enrollment_df):
    """Process object data and create features."""
    # Count the number of modules per course
    modules_per_course = object_df.groupby('course_id').size().reset_index(name='module_count')
    
    # Count the number of each category per course
    category_counts = object_df.groupby(['course_id', 'category']).size().unstack(fill_value=0)
    category_counts.columns = [f'category_{col}_count' for col in category_counts.columns]
    
    # Merge all object features
    object_features = pd.merge(modules_per_course, category_counts, on='course_id', how='left')
    
    return pd.merge(enrollment_df, object_features, on='course_id', how='left')


In [84]:
def aggregate_log_data(log):
    """Aggregate log data to create features."""
    return log.groupby('enrollment_id').agg({
        'source': 'count',
        'event': lambda x: x.nunique(),
        'object': 'nunique'
    }).rename(columns={
        'source': 'activity_count',
        'event': 'unique_events',
        'object': 'unique_objects'
    })

def merge_datasets(enrollment, log_features, truth=None):
    """Merge the preprocessed datasets."""
    df = pd.merge(enrollment, log_features, on='enrollment_id', how='left')
    df.fillna(0, inplace=True)
    if truth is not None:
        df = pd.merge(df, truth, on='enrollment_id')
    return df

def engineer_features(df):
    """Perform feature engineering."""
    df['event_object_ratio'] = df['unique_events'] / (df['unique_objects'] + 1)
    df['activity_per_event'] = df['activity_count'] / (df['unique_events'] + 1)
    return df

def prepare_features(df):
    """Prepare features for model training or prediction."""
    X = df.drop(['enrollment_id', 'dropout','username','course_id'], axis=1)
    y = df['dropout']
    return X, y


def engineer_additional_features(df):
    """Engineer additional features using the new data."""
    df['modules_per_day'] = df['module_count'] / df['course_duration']
    df['category_diversity'] = df[[col for col in df.columns if col.startswith('category_')]].gt(0).sum(axis=1)
    return df


In [85]:
def train_model(X, y):
    """Train the Random Forest model."""
    model = RandomForestClassifier(n_estimators=50, random_state=42)
    model.fit(X, y)
    return model

def evaluate_model(model, X, y):
    """Evaluate the model and return performance metrics."""
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    accuracy = accuracy_score(y, y_pred)
    auc = roc_auc_score(y, y_pred_proba)
    return accuracy, auc

In [73]:
def calculate_metrics(y_true, y_pred, y_pred_proba):
    """Calculate accuracy, AUC, precision, and recall."""
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return accuracy, auc, precision, recall

In [86]:
train_enrollment, train_log, train_truth = load_data('train')
test_enrollment, test_log, test_truth = load_data('test')

train_enrollment = preprocess_enrollment(train_enrollment)
test_enrollment = preprocess_enrollment(test_enrollment)

train_log_features = aggregate_log_data(train_log)
test_log_features = aggregate_log_data(test_log)

train_df = merge_datasets(train_enrollment, train_log_features, train_truth)
test_df = merge_datasets(test_enrollment, test_log_features, test_truth)


        # Pro

In [87]:
train_df.head()

,enrollment_id,username,course_id,activity_count,unique_events,unique_objects,dropout
0,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,314,5,95,0
1,3,1qXC7Fjbwp66GPQc6pHLfEuO8WKozxG4,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,288,6,99,0
2,4,FIHlppZyoq8muPbdVxS44gfvceX9zvU7,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,99,5,29,0
3,5,p1Mp7WkVfzUijX0peVQKSHbgd5pXyl4c,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,633,6,111,0
4,6,dpK33RH9yepUAnyoywRwBt1AJzxGlaja,AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd,23,5,16,0


In [88]:
def main():
    try:
        # Load all data
        train_enrollment, train_log, train_truth = load_data('train')
        test_enrollment, test_log, test_truth = load_data('test')
        #date_df, object_df = load_additional_data()

        # Preprocess enrollment data
        train_enrollment = preprocess_enrollment(train_enrollment)
        test_enrollment = preprocess_enrollment(test_enrollment)

        # Process date and object data
        #train_enrollment = process_date_data(date_df, train_enrollment)
        #train_enrollment = process_object_data(object_df, train_enrollment)
        #test_enrollment = process_date_data(date_df, test_enrollment)
        #test_enrollment = process_object_data(object_df, test_enrollment)

        # Aggregate log data
        train_log_features = aggregate_log_data(train_log)
        test_log_features = aggregate_log_data(test_log)

        # Merge datasets
        train_df = merge_datasets(train_enrollment, train_log_features, train_truth)
        test_df = merge_datasets(test_enrollment, test_log_features, test_truth)

        # Engineer features
        train_df = engineer_features(train_df)
        test_df = engineer_features(test_df)
        #train_df = engineer_additional_features(train_df)
        #test_df = engineer_additional_features(test_df)

        # Prepare features
        X_train, y_train = prepare_features(train_df)
        X_test, y_test = prepare_features(test_df)

        # Train the model
        model = train_model(X_train, y_train)

        # Evaluate on training data
        y_train_pred = model.predict(X_train)
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]
        train_metrics = calculate_metrics(y_train, y_train_pred, y_train_pred_proba)
        
        print("Training Metrics:")
        print(f"Accuracy: {train_metrics[0]:.4f}")
        print(f"AUC: {train_metrics[1]:.4f}")
        print(f"Precision: {train_metrics[2]:.4f}")
        print(f"Recall: {train_metrics[3]:.4f}")

        # Evaluate on test data
        y_test_pred = model.predict(X_test)
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]
        test_metrics = calculate_metrics(y_test, y_test_pred, y_test_pred_proba)
        
        print("\nTest Metrics:")
        print(f"Accuracy: {test_metrics[0]:.4f}")
        print(f"AUC: {test_metrics[1]:.4f}")
        print(f"Precision: {test_metrics[2]:.4f}")
        print(f"Recall: {test_metrics[3]:.4f}")

       
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Stack trace:")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Training Metrics:
Accuracy: 0.9083
AUC: 0.9018
Precision: 0.9102
Recall: 0.9813

Test Metrics:
Accuracy: 0.8331
AUC: 0.7814
Precision: 0.8689
Recall: 0.9295

Predictions saved to submission.csv


In [89]:
F1 = 2*0.86*0.93/(0.86+0.93)

In [90]:
F1

0.8936312849162011